In [ ]:
# Flight Experience
# Education
# Skills
# Interest
# Certification
# Competences
# Languages
# WorkExperiences
# Survey


In [ ]:
#------------------------------------------------------------------------------#
# Importing packages                                                           #
#------------------------------------------------------------------------------#
from pyspark.sql import Window
from pyspark.sql.functions import arrays_zip, explode
from pyspark.sql import functions as F
from pyspark.sql.functions import col
# added to replace and cast column
from pyspark.sql.functions import *
# added to replace and cast column
# add to apply sorting
from pyspark.sql.functions import lit
# add to apply sorting

import os
import json
import pyspark
import glob
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
from pyspark.sql.types import (
    StructType,
    StructField,
    TimestampType,
    StringType,
    BooleanType,
    IntegerType,
    DateType,
    LongType
)

from pyspark.sql.types import (
    TimestampType
)
from pyspark.sql.functions import (

    col, when, size, split, unix_timestamp,
    from_unixtime

)

import pyspark.sql.functions as func



In [ ]:
#------------------------------------------------------------------------------#
# Table with FlightExperiences information                                     #
#------------------------------------------------------------------------------#
url='abfss://raw@datalakeprd.dfs.core.windows.net/FlightExperiences/v1/*/*/*.blob'
df_FlightExperiences = spark.read.json(url)

#------------------------------------------------------------------------------#
# Table with Education information                                             #
#------------------------------------------------------------------------------#
url='abfss://raw@datalakeprd.dfs.core.windows.net/Educations/v1/*/*/*.blob'
df_education = spark.read.json(url)


#------------------------------------------------------------------------------#
# Table with Skills information                                                #
#------------------------------------------------------------------------------#
url='abfss://raw@datalakeprd.dfs.core.windows.net/Skills/v1/*/*/*.blob'
df_Skills = spark.read.json(url)

#------------------------------------------------------------------------------#
# Table with WorkExperiences information                                       #
#------------------------------------------------------------------------------#
url='abfss://raw@datalakeprd.dfs.core.windows.net/WorkExperiences/v1/*/*/*.blob'
df_WorkExperiences = spark.read.json(url)

#------------------------------------------------------------------------------#
# Table with Survey Feedback on Airside information                            #
#------------------------------------------------------------------------------#
url='abfss://raw@datalakeprd.dfs.core.windows.net/Feedback/v1/*/*/*.blob'
df_SurveyFeedback = spark.read.json(url)

#------------------------------------------------------------------------------#
# Table with FlightLicenses information                            #
#------------------------------------------------------------------------------#
url='abfss://raw@datalakeprd.dfs.core.windows.net/FlightLicenses/v1/*/*/*.blob'
df_FlightLicenses = spark.read.json(url)

In [ ]:
#------------------------------------------------------------------------------#
# Table Users LICENSES                                                         #
# with nested content                                                          #
#------------------------------------------------------------------------------#

FlightLicenses  = df_FlightLicenses.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  col('Payload.UserId').alias('UserId'),
  col('Payload.id').alias('id'),
  col('Payload.Rank').alias('Rank'),
  col('Type').alias('FL_Type'),
  col('Version').alias('FL_Version')
  )
  
  
InstrumentRatings  = df_FlightLicenses.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  explode('Payload.InstrumentRatings').alias('InstrumentRatings'), 
  col('Payload.UserId').alias('UserId'),
  col('Payload.id').alias('id')
  )

InstrumentRatings  = InstrumentRatings.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  col('InstrumentRatings.Aircraft').alias('Aircraft'),
  col('InstrumentRatings.ExpiryDate').alias('ExpiryDate'),
  col('UserId').alias('UserId'),
  col('id').alias('id')
  )

# Licenses
Licenses  = df_FlightLicenses.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  explode('Payload.Licenses').alias('Licenses'), 
  col('Payload.UserId').alias('UserId'),
  col('Payload.id').alias('id')
  )

Licenses  = Licenses.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  
  col('Licenses.Country.Alpha2Code').alias('Country'),
  func.to_date('Licenses.ExpiresOn').alias('ExpiryDate'),
  col('Licenses.Type.Code').alias('Code'),
  col('Licenses.Type.Description').alias('Description'),
  
  col('UserId').alias('UserId'),
  col('id').alias('id')
  )

# Qualifications
Qualifications  = df_FlightLicenses.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  explode('Payload.Qualifications').alias('Qualifications'), 
  col('Payload.UserId').alias('UserId'),
  col('Payload.id').alias('id')
  )

Qualifications  = Qualifications.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  
  col('Qualifications.Title').alias('Title'),
  func.to_date('Qualifications.ExpiryDate').alias('ExpiryDate'),
  
  col('UserId').alias('UserId'),
  col('id').alias('id')
  )

# TypeRatings
TypeRatings  = df_FlightLicenses.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  explode('Payload.TypeRatings').alias('TypeRatings'), 
  col('Payload.UserId').alias('UserId'),
  col('Payload.id').alias('id')
  )

TypeRatings  = TypeRatings.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  
  col('TypeRatings.Country.Alpha2Code').alias('Country'),
  func.to_date('TypeRatings.DateOfLastFlight').alias('DateOfLastFlight'),
  func.to_date('TypeRatings.LpcDateOfIssue').alias('LpcDateOfIssue'),
  col('TypeRatings.Name').alias('Name'),
  func.to_date('TypeRatings.OpcDateOfIssue').alias('OpcDateOfIssue'),
  col('TypeRatings.Regulatory').alias('Regulatory'),
  func.to_date('TypeRatings.TrainingCompletionDate').alias('TrainingCompletionDate'),
  
  col('UserId').alias('UserId'),
  col('id').alias('id')
  )


FlightLicenses = FlightLicenses.distinct()
InstrumentRatings = InstrumentRatings.distinct()
Licenses = Licenses.distinct()
Qualifications = Qualifications.distinct()
TypeRatings = TypeRatings.distinct()

In [ ]:
#------------------------------------------------------------------------------#
# Table with Users Flights Experiences                                         #
# with nested content                                                          #
#------------------------------------------------------------------------------#
#df_users.printSchema()
FlightsExperiences  = df_FlightExperiences.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  explode('Payload.FlightHours').alias('FlightHours'),
  col('Payload.UserId').alias('FE_UserId'),
  col('Payload.id').alias('FE_id') 
  )

FlightsExperiencesDetails = FlightsExperiences.select(
  col('EventEnqueuedUtcTime').alias('EventEnqueuedUtcTime'),
  col('EventProcessedUtcTime').alias('EventProcessedUtcTime'),
  col('FlightHours.Aircraft').alias('FE_Aircraft'),
  col('FlightHours.AircraftCategory').alias('FE_AircraftCategory'),
  col('FlightHours.DateOfLastFlight').alias('FE_DateOfLastFlight'),
  col('FlightHours.Position').alias('FE_Position'),
  col('FlightHours.TotalFlightHours').alias('FE_TotalFlightHours'),
  col('FE_UserId').alias('FE_UserId'),
  col('FE_id').alias('FE_id')
                        )
FlightsExperiencesDetails = FlightsExperiencesDetails.distinct()


In [ ]:
#------------------------------------------------------------------------------#
# Table with Users Education                                                   #
# with nested content                                                          #
#------------------------------------------------------------------------------#

Education  = df_education.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  col('Payload.HasCompletedPostSecondaryLevel').alias('UE_HasCompletedPostSecondaryLevel'),
  explode('Payload.Studies').alias('Studies'),
  col('Payload.UserId').alias('UE_UserId'),
  col('Payload.id').alias('UE_id') 
  )


EducationDetails = Education.select(
  col('EventEnqueuedUtcTime').alias('EventEnqueuedUtcTime'),
  col('EventProcessedUtcTime').alias('EventProcessedUtcTime'),
  col('UE_HasCompletedPostSecondaryLevel').alias('UE_HasCompletedPostSecondaryLevel'),
  
  col('Studies.Degree').alias('UE_Degree'),
  col('Studies.Description').alias('UE_Description'),
  col('Studies.EndDate').alias('UE_EndDate'),
  col('Studies.FieldOfStudy').alias('UE_FieldOfStudy'),
  col('Studies.IsAviationRelated').alias('UE_IsAviationRelated'),
  col('Studies.SchoolName').alias('UE_SchoolName'),
  col('Studies.StartDate').alias('UE_StartDate'),  
  col('UE_UserId').alias('UE_UserId'),
  col('UE_id').alias('UE_id')
                         )
EducationDetails = EducationDetails.distinct()


In [ ]:
#------------------------------------------------------------------------------#
# Table Users Skills                                                           #
# with nested content                                                          #
#------------------------------------------------------------------------------#

Skills  = df_Skills.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  col('Payload.ProfessionalSummary').alias('US_ProfessionalSummary'), 
  col('Payload.UserId').alias('US_UserId'),
  col('Payload.id').alias('US_id') 
  )

Interests  = df_Skills.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  explode('Payload.Interests.Name').alias('US_Interests'), 
  col('Payload.UserId').alias('US_UserId'),
  col('Payload.id').alias('US_id') 
  )

Certification  = df_Skills.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  explode('Payload.Certifications').alias('Certifications'),
  col('Payload.UserId').alias('US_UserId'),
  col('Payload.id').alias('US_id') 
  )

Competences = df_Skills.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  explode('Payload.Competences').alias('Competences'),
  col('Payload.UserId').alias('US_UserId'),
  col('Payload.id').alias('US_id') 
  )

Languages = df_Skills.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  explode('Payload.Languages').alias('Languages'),
  col('Payload.UserId').alias('US_UserId'),
  col('Payload.id').alias('US_id') 
  )

CertificationDetails  = Certification.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  col('Certifications.IssuedON').alias('CER_IssuedON'),
  col('Certifications.Name').alias('CER_Name'),
  col('US_UserId').alias('US_UserId'),
  col('US_id').alias('US_id') 
)

CompetencesDetails = Competences.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  col('Competences.Name').alias('COM_Name'),
  col('US_UserId').alias('US_UserId'),
  col('US_id').alias('US_id') 
  )

LanguagesDetails = Languages.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  col('Languages.ExpiryDate').alias('LAN_ExpiryDate'),
  col('Languages.Name').alias('LAN_Name'),
  col('Languages.Proficiency.Code').alias('LAN_ProficiencyCode'),
  col('Languages.Proficiency.Description').alias('LAN_ProficiencyDescription'),
  col('US_UserId').alias('US_UserId'),
  col('US_id').alias('US_id') 
  )

Skills = Skills.distinct()
CertificationDetails = CertificationDetails.distinct()
CompetencesDetails = CompetencesDetails.distinct()
LanguagesDetails = LanguagesDetails.distinct()
Interests = Interests.distinct()


In [ ]:
#------------------------------------------------------------------------------#
# Table Users df_WorkExperiences                                               #
# with nested content                                                          #
#------------------------------------------------------------------------------#

WorkExperiences  = df_WorkExperiences.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  col('Payload.HasExperiences').alias('WE_HasExperiences'), 
  col('Payload.UserId').alias('WE_UserId'),
  col('Payload.id').alias('WE_id') 
  )

Experiences  = df_WorkExperiences.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  explode('Payload.Experiences').alias('Experiences'),
  col('Payload.UserId').alias('WE_UserId'),
  col('Payload.id').alias('WE_id') 
  )
WorkExperiencesDetails  = Experiences.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  col('Experiences.Aircraft').alias('WE_Aircraft'),
  col('Experiences.City').alias('WE_City'),
  col('Experiences.CountryAlpha2Code').alias('WE_CountryAlpha2Code'),
  col('Experiences.Description').alias('WE_Description'),
  col('Experiences.Employer').alias('WE_Employer'),
  col('Experiences.EmploymentType').alias('WE_EmploymentType'),
  col('Experiences.EndDate').alias('WE_EndDate'),
  col('Experiences.IsCurrentRole').alias('WE_IsCurrentRole'),
  col('Experiences.JobTitle').alias('WE_JobTitle'),
  col('Experiences.Position').alias('WE_Position'),
  col('Experiences.StartDate').alias('WE_StartDate'),
  col('WE_UserId').alias('WE_UserId'),
  col('WE_id').alias('WE_id') 
  )

WorkExperiencesDetails = WorkExperiencesDetails.distinct()

In [ ]:

#------------------------------------------------------------------------------#
# Table Users df_SurveyFeedback                                                #
# with nested content                                                          #
#------------------------------------------------------------------------------#

SurveyFeedback  = df_SurveyFeedback.select(
  func.to_date('EventEnqueuedUtcTime').alias("EventEnqueuedUtcTime"),
  func.to_date('EventProcessedUtcTime').alias("EventProcessedUtcTime"),
  col('Payload.Answer.DisplayText').alias('SF_AnswerDisplayText'),
  col('Payload.Answer.Key').alias('SF_AnswerKey'),
  col('Payload.Answer.OtherText').alias('SF_AnswerOtherText'),
  col('Payload.DisplayText').alias('SF_DisplayText'),
  col('Payload.Key').alias('SF_Key'),
  col('Payload.Tags').getItem(0).alias('SF_Tags')  
  )

SurveyFeedback = SurveyFeedback.distinct()


In [ ]:
# get latest itteration for each record for all tables
FlightsExperiencesDetails.createOrReplaceTempView("FlightsExperiencesDetails")
FlightsExperiencesDetails_curated =spark.sql("select  * from (\
                                          SELECT distinct *, \
                                          EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY FE_UserId) as max_date\
                                          FROM FlightsExperiencesDetails ) as t\
                          where t.max_date = True\
                          ")

EducationDetails.createOrReplaceTempView("EducationDetails")
EducationDetails_curated =spark.sql("select  * from (\
                                          SELECT distinct *, \
                                          EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY UE_UserId) as max_date\
                                          FROM EducationDetails ) as t\
                          where t.max_date = True\
                          ")

Skills.createOrReplaceTempView("Skills")
Skills_curated =spark.sql("select  * from (\
                                          SELECT distinct *, \
                                          EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY US_UserId) as max_date\
                                          FROM Skills ) as t\
                          where t.max_date = True\
                          ")

CertificationDetails.createOrReplaceTempView("CertificationDetails")
CertificationDetails_curated =spark.sql("select  * from (\
                                          SELECT distinct *, \
                                          EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY US_UserId) as max_date\
                                          FROM CertificationDetails ) as t\
                          where t.max_date = True\
                          ")

CompetencesDetails.createOrReplaceTempView("CompetencesDetails")
CompetencesDetails_curated =spark.sql("select  * from (\
                                          SELECT distinct *, \
                                          EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY US_UserId) as max_date\
                                          FROM CompetencesDetails ) as t\
                          where t.max_date = True\
                          ")

LanguagesDetails.createOrReplaceTempView("LanguagesDetails")
LanguagesDetails_curated =spark.sql("select  * from (\
                                          SELECT distinct *, \
                                          EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY US_UserId) as max_date\
                                          FROM LanguagesDetails ) as t\
                          where t.max_date = True\
                          ")

Interests.createOrReplaceTempView("Interests")
Interests_curated =spark.sql("select  * from (\
                                          SELECT distinct *, \
                                          EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY US_UserId) as max_date\
                                          FROM Interests ) as t\
                          where t.max_date = True\
                          ")

WorkExperiencesDetails.createOrReplaceTempView("WorkExperiencesDetails")
WorkExperiencesDetails_curated =spark.sql("select  * from (\
                                          SELECT distinct *, \
                                          EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY WE_UserId) as max_date\
                                          FROM WorkExperiencesDetails ) as t\
                          where t.max_date = True\
                          ")

SurveyFeedback.createOrReplaceTempView("SurveyFeedback")
SurveyFeedback_curated =spark.sql("select  * from (\
                                          SELECT distinct *, \
                                          EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY SF_Key) as max_date\
                                          FROM SurveyFeedback ) as t\
                          where t.max_date = True\
                          ")

In [ ]:
# get latest itteration for each record for all tables
FlightLicenses.createOrReplaceTempView("FlightLicenses")
FlightLicenses_curated =spark.sql("""select  * from (
                                                      SELECT distinct *, 
                                                      EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY UserId) as max_date
                                                      FROM FlightLicenses ) as t
                                             where t.max_date = True
                          """)

InstrumentRatings.createOrReplaceTempView("InstrumentRatings")
InstrumentRatings_curated =spark.sql("""select  * from (
                                                      SELECT distinct *, 
                                                      EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY UserId) as max_date
                                                      FROM InstrumentRatings ) as t
                                             where t.max_date = True
                          """)

Licenses.createOrReplaceTempView("Licenses")
Licenses_curated =spark.sql("""select  * from (
                                                      SELECT distinct *, 
                                                      EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY UserId) as max_date
                                                      FROM Licenses ) as t
                                             where t.max_date = True
                          """)

Qualifications.createOrReplaceTempView("Qualifications")
Qualifications_curated =spark.sql("""select  * from (
                                                      SELECT distinct *, 
                                                      EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY UserId) as max_date
                                                      FROM Qualifications ) as t
                                             where t.max_date = True
                          """)

TypeRatings.createOrReplaceTempView("TypeRatings")
TypeRatings_curated =spark.sql("""select  * from (
                                                      SELECT distinct *, 
                                                      EventProcessedUtcTime = MAX(EventProcessedUtcTime) OVER (PARTITION BY UserId) as max_date
                                                      FROM TypeRatings ) as t
                                             where t.max_date = True
                          """)


# 

In [ ]:
#create empty columns
FlightsExperiencesDetails_curated =FlightsExperiencesDetails_curated.withColumn('Aircraft',lit('Unidentified').cast(StringType()))
FlightsExperiencesDetails_curated =FlightsExperiencesDetails_curated.withColumn('AircraftManufacturer',lit('Unidentified').cast(StringType()))

#fille UserAirccraft Dictionary in a dataframe and sort
usersaircraftentry = spark.sql("select * from nasa_usersaircraftentry")
usersaircraftentry.orderBy(asc('Priority'),asc('SearchKey'))

In [ ]:
#Clean Aircraft Data from free entry field
for uaerow in usersaircraftentry.rdd.collect():
  FlightsExperiencesDetails_curated = FlightsExperiencesDetails_curated.withColumn(
                                                                                "Aircraft",
                                                                                F.when((F.lower(F.col("FE_Aircraft")).contains(uaerow.SearchKey.lower())) & (F.col("Aircraft")=='Unidentified'),
                                                                                      uaerow.Aircraft).otherwise(F.col("Aircraft"))
                                                                                      )
  FlightsExperiencesDetails_curated = FlightsExperiencesDetails_curated.withColumn(
                                                                              "AircraftManufacturer",
                                                                              F.when((F.lower(F.col("FE_Aircraft")).contains(uaerow.SearchKey.lower())) & (F.col("AircraftManufacturer")=='Unidentified'),
                                                                              uaerow.AircraftFamily).otherwise(F.col("AircraftManufacturer"))
                                                                            )

In [ ]:
#------------------------------------------------------------------------------#
#  DROP and SAVE a tables                                                      #
#------------------------------------------------------------------------------#
#FlightsExperiencesDetails
spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_FlightsExperiencesDetails")
FlightsExperiencesDetails_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_FlightsExperiencesDetails")

#EducationDetails
spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_EducationDetails")
EducationDetails_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_EducationDetails")

#Skills 
spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_Skills")
Skills_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_Skills")

#interests 
spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_interests")
Interests_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_interests")

#CertificationDetails 
spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_CertificationDetails")
CertificationDetails_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_CertificationDetails")

#CompetencesDetails 
spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_CompetencesDetails")
CompetencesDetails_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_CompetencesDetails")

#LanguagesDetails
spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_LanguagesDetails")
LanguagesDetails_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_LanguagesDetails")

#WorkExperiencesDteail
spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_WorkExperiencesDetails")
WorkExperiencesDetails_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_WorkExperiencesDetails")

#SurveyFeedback
spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_SurveyFeedback")
SurveyFeedback_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_SurveyFeedback")


In [ ]:
#SurveyFeedback
spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_FlightLicenses")
FlightLicenses_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_FlightLicenses")

spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_InstrumentRatings")
InstrumentRatings_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_InstrumentRatings")


spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_Licenses")
Licenses_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_Licenses")

spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_Qualifications")
Qualifications_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_Qualifications")

spark.sql("DROP TABLE IF EXISTS cx_pilot.airside_users_TypeRatings")
TypeRatings_curated.write.format("delta").saveAsTable("cx_pilot.airside_users_TypeRatings")



In [ ]:
%sql
REFRESH TABLE cx_pilot.airside_users_FlightsExperiencesDetails;
REFRESH TABLE cx_pilot.airside_users_EducationDetails;
REFRESH TABLE cx_pilot.airside_users_Skills;
REFRESH TABLE cx_pilot.airside_users_interests;
REFRESH TABLE cx_pilot.airside_users_CertificationDetails;
REFRESH TABLE cx_pilot.airside_users_CompetencesDetails;
REFRESH TABLE cx_pilot.airside_users_LanguagesDetails;
REFRESH TABLE cx_pilot.airside_users_WorkExperiencesDetails;
REFRESH TABLE cx_pilot.airside_users_SurveyFeedback;

REFRESH TABLE  cx_pilot.airside_users_FlightLicenses;
REFRESH TABLE  cx_pilot.airside_users_InstrumentRatings;
REFRESH TABLE  cx_pilot.airside_users_Licenses;
REFRESH TABLE  cx_pilot.airside_users_Qualifications;
REFRESH TABLE  cx_pilot.airside_users_TypeRatings;

In [ ]:
%sql
optimize cx_pilot.airside_users_FlightsExperiencesDetails;
optimize cx_pilot.airside_users_EducationDetails;
optimize cx_pilot.airside_users_Skills;
optimize cx_pilot.airside_users_interests;
optimize cx_pilot.airside_users_CertificationDetails;
optimize cx_pilot.airside_users_CompetencesDetails;
optimize cx_pilot.airside_users_LanguagesDetails;
optimize cx_pilot.airside_users_WorkExperiencesDetails;
optimize cx_pilot.airside_users_SurveyFeedback;


Optimize cx_pilot.airside_users_FlightLicenses;
Optimize cx_pilot.airside_users_InstrumentRatings;
Optimize cx_pilot.airside_users_Licenses;
Optimize cx_pilot.airside_users_Qualifications;
Optimize cx_pilot.airside_users_TypeRatings;
